# 사전학습된 모델을 사용한 전이학습
사전학습된 **Xception** 모델을 사용하여 꽃 이미지를 분류하는 모델 학습하기

In [3]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='C:\\Users\\gus8c\\tensorflow_datasets\\tf_flowers\\3.0.1',
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

In [4]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

In [5]:
train_set = tfds.load("tf_flowers", split='train[:70%]', as_supervised=True)
valid_set = tfds.load("tf_flowers", split='train[70%:85%]', as_supervised=True)
test_set = tfds.load("tf_flowers", split='train[85%:100%]', as_supervised=True)

In [6]:
len(train_set), len(valid_set), len(test_set)

(2569, 551, 550)

In [1]:
import tensorflow as tf
from tensorflow import keras

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [7]:
batch_size = 16

train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

> 데이터 증식을 추가하고 싶다면 **preprocess** 함수에  
**tf.image.random_crop()** : 이미지를 랜덤하게 자르거나,  
**tf.image.random_flip_left_right** : 이미지를 수평으로 랜덤하게 뒤집는다거나  
여러가지 증강을 추가할 수 있음

In [8]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)

model = keras.Model(inputs=base_model.input, outputs=output)

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
opt = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [11]:
import os
from datetime import datetime
logs = os.path.join(os.curdir, "my_flower_logs",
                    "run_" + datetime.now().strftime("%Y%m%d_%H%M%S"))

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir = logs)
early_cb = tf.keras.callbacks.EarlyStopping(patience=20)


model.fit(train_set, epochs=5, validation_data=valid_set, callbacks=[tensorboard_cb, early_cb])

Epoch 1/5
161/161 [==============================] - 18s 87ms/step - loss: 3.0388 - accuracy: 0.7131 - val_loss: 0.8682 - val_accuracy: 0.8838
Epoch 2/5
161/161 [==============================] - 11s 70ms/step - loss: 0.8601 - accuracy: 0.8903 - val_loss: 1.0317 - val_accuracy: 0.8639
Epoch 3/5
161/161 [==============================] - 11s 70ms/step - loss: 0.3633 - accuracy: 0.9349 - val_loss: 0.7811 - val_accuracy: 0.8893
Epoch 4/5
161/161 [==============================] - 12s 72ms/step - loss: 0.2277 - accuracy: 0.9513 - val_loss: 0.6803 - val_accuracy: 0.8820
Epoch 5/5
161/161 [==============================] - 11s 70ms/step - loss: 0.1268 - accuracy: 0.9639 - val_loss: 0.6377 - val_accuracy: 0.8857


In [12]:
keras.backend.clear_session()
for layer in base_model.layers:
    layer.trainable = True
    
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

    
model.fit(train_set, epochs=100, validation_data=valid_set, callbacks=[tensorboard_cb, early_cb], initial_epoch=5)

Epoch 6/100


ResourceExhaustedError:  OOM when allocating tensor with shape[16,128,109,109] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/block2_sepconv2/separable_conv2d (defined at <ipython-input-12-ac1fba2f1493>:8) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_24374]

Function call stack:
train_function


In [ ]:
for layer in base_model.layers:
    print(layer.trainable)